In [1]:
from pyspark.sql import SparkSession
from sparkmeasure import StageMetrics
from pyspark.sql.functions import lower, col

spark = SparkSession.builder.appName('q2').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()
#For VIRTUAL
#spark = SparkSession.builder.master('spark://picachu-VirtualBox:7077').appName('q2').config("spark.jars", "/home/athina/Downloads/spark-measure_2.12-0.17.jar").getOrCreate()

#For LIVY
#spark=SparkSession.builder.master('spark://localhost:7077').appName("Testing").getOrCreate()
#and changed the filepaths to '/home/administrator/Downloads/movielens/file_name.csv'

stagemetrics = StageMetrics(spark)
stagemetrics.begin()#start measuring performance

df_tag= (spark.read
     .format("csv")
     .option('header', 'true')
     .option('delimiter', ",")
     .option("inferSchema","true")
     .load("tag.csv") 
    )
df_movie= (spark.read
     .format("csv")
     .option('header', 'true')
     .option('delimiter', ",")
     .option("inferSchema","true")
     .load("movie.csv") 
    )

lc=df_tag.select('*',lower(col('tag')).alias('tag_lc')) #get low case tags
boring=lc.filter(lc.tag_lc.like("boring%")).select(lc["movieId"]) #all kinds of boring
#boring2=lc.where(lc.tag_lc.like("boring")|lc.tag_lc.like("boring_")).select(lc["movieId"]) # "boring" or "boring!"

results=df_movie.join(boring, boring["movieId"]==df_movie["movieId"],"inner").hint("broadcast").select(df_movie["title"]).distinct()
results.orderBy("title").show(5)#requested answer for query 2

stagemetrics.end()#stop measuring performance
stagemetrics.print_report()#print performance metrics
spark.stop()#stop spark

+--------------------+
|               title|
+--------------------+
|(500) Days of Sum...|
|101 Reykjavik (10...|
|12 Years a Slave ...|
|         1408 (2007)|
|1492: Conquest of...|
+--------------------+
only showing top 5 rows


Scheduling mode = FIFO
Spark Context default degree of parallelism = 2
Aggregated Spark stage metrics:
numStages => 7
numTasks => 208
elapsedTime => 9949 (10 s)
stageDuration => 9024 (9 s)
executorRunTime => 8783 (9 s)
executorCpuTime => 7334 (7 s)
executorDeserializeTime => 3679 (4 s)
executorDeserializeCpuTime => 2802 (3 s)
resultSerializationTime => 48 (48 ms)
jvmGCTime => 419 (0.4 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 63 (63 ms)
resultSize => 1408093 (1375.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 857735168
recordsRead => 985688
bytesRead => 46732906 (44.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 767
shuffleTotalBlocksFetched => 337
shuffleLocalBlocks